In [1]:
# Download model, and save
# https://huggingface.co/kakao-enterprise/vits-ljs
from transformers import VitsModel, AutoTokenizer
import torch

models_folder_path = "./../models"
model_name = "kakao-enterprise/vits-ljs"
model = VitsModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained(models_folder_path+ "/local_vits")
tokenizer.save_pretrained(models_folder_path+ "/local_vits")

2025-04-04 04:40:59.545201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743734459.641191  174133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743734459.668091  174133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743734459.884240  174133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743734459.884277  174133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743734459.884280  174133 computation_placer.cc:177] computation placer alr

('./../models/local_vits/tokenizer_config.json',
 './../models/local_vits/special_tokens_map.json',
 './../models/local_vits/vocab.json',
 './../models/local_vits/added_tokens.json')

In [ ]:
from transformers import VitsModel, AutoTokenizer
import time

start = time.time()
text = "specifies."
text += "specifies the required processes implemented in the engineering activities."
# text += "specifies the required processes implemented in the engineering activities that result in requirements for systems and software products (including services) throughout the life cycle"
# text += "provides guidelines for applying the requirements and requirements-related processes described in ISO/IEC/IEEE 15288 and ISO/IEC/IEEE 12207;"
text = text.replace("(", "")
text = text.replace(")", "")

print(time.time() - start)
inputs = tokenizer(text, return_tensors="pt")

print(time.time() - start)
with torch.no_grad():
    output = model(**inputs).waveform

print(time.time() - start)

Audio(output, rate=model.config.sampling_rate)
model = VitsModel.from_pretrained("./local_vits")
tokenizer = AutoTokenizer.from_pretrained("./local_vits")